<a href="https://colab.research.google.com/github/SiddhantIQByte/Web-Scrapping-Tutorial-and-Scrapped-Dataset/blob/main/IMDB_MOVIE_DATA_SCRAPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***IMDB MOVIE DATA SCRAPING***

**Import Nessecary Modules:**

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from random import randint
import re

**Acessing the website:**

In [15]:

imdb_url = 'https://www.imdb.com/list/ls050782187/?sort=list_order,asc&st_dt=&mode=detail&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
response = requests.get(imdb_url, headers=headers)
response.status_code

200

In [16]:
soup = BeautifulSoup(response.text, 'lxml')

In [17]:
movies = soup.find_all('div', class_='lister-item-content')
for movie in movies[:5]:
    print(movie.find('h3', class_='lister-item-header').a.text.strip())

The Godfather
The Silence of the Lambs
Star Wars: Episode V - The Empire Strikes Back
The Shawshank Redemption
The Shining


In [31]:
top_movies = []

for i in range(1,6):

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    url = f'https://www.imdb.com/list/ls050782187/?st_dt=&mode=detail&page={i}&sort=list_order,asc'

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content, 'lxml')

    time.sleep(randint(0,3))
    all_movies = soup.find_all('div', class_='lister-item-content')
    for movie in all_movies:

        # movie title
        title = movie.find('h3').a.text.strip()

        # Release Year
        rl = movie.find('span', class_='lister-item-year text-muted unbold')
        release = rl.text.strip()
        release_year =re.sub(r'[{\[()\]}]', '', release) if rl else np.nan
        # Director
        dir_name = movie.find_all('p', class_='text-muted text-small')[1].find_all('a')[0]
        director = (dir_name.text if dir_name else np.nan)
        # Genre
        gnr = movie.find('span', class_='genre')
        genre = gnr.text.strip() if gnr else np.nan

        # Film Rating
        fr = movie.find('span', class_='certificate')
        film_rating = fr.text.strip() if fr else np.nan

        # Runtime
        rt = movie.find('span', class_='runtime')
        runtime = rt.text.strip() if rt else np.nan


        # IMDb rating
        imdb = movie.find('span', class_='ipl-rating-star__rating')
        imdb_rating = imdb.text.strip() if imdb else np.nan

        # Metascore
        mscore = movie.find('div', class_='inline-block ratings-metascore')
        metascore = mscore.span.text if mscore else np.nan

        # Description
        desc =  movie.find_all('p')[1]
        description = desc.text.strip() if desc else np.nan



        # Votes
        vts = movie.find_all('p', class_='text-muted text-small')[2].find_all('span')[1]
        votes = vts.text if vts else np.nan

        #Gross
        grs = movie.find_all('p', class_='text-muted text-small')[2].find_all('span')[-1]
        gross = grs.text if grs else np.nan


        # append all the data in to a 2D list
        top_movies.append([title, release_year,director,genre, film_rating, runtime,  imdb_rating, metascore, description,  votes, gross])


    print(f'Complete Page: {i}')

print('All Done!')


Complete Page: 1
Complete Page: 2
Complete Page: 3
Complete Page: 4
Complete Page: 5
All Done!


In [32]:

columns = ['Movie_Title', 'Release_Year','Director','Genre', 'Film_Rating', 'Runtime',  'imdb_rating', 'metascore', 'description',  'votes', 'gross' ]


data = pd.DataFrame(top_movies, columns=columns)

In [33]:
data.head()

,Movie_Title,Release_Year,Director,Genre,Film_Rating,Runtime,imdb_rating,metascore,description,votes,gross
0,The Godfather,1972,Francis Ford Coppola,"Crime, Drama",R,175 min,9.2,100,The aging patriarch of an organized crime dyna...,"1,993,054",$134.97M
1,The Silence of the Lambs,1991,Jonathan Demme,"Crime, Drama, Thriller",R,118 min,8.6,86,A young F.B.I. cadet must receive the help of ...,"1,533,346",$130.74M
2,Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,"Action, Adventure, Fantasy",PG,124 min,8.7,82,After the Rebels are overpowered by the Empire...,"1,368,072",$290.48M
3,The Shawshank Redemption,1994,Frank Darabont,Drama,R,142 min,9.3,82,"Over the course of several years, two convicts...","2,861,124",$28.34M
4,The Shining,1980,Stanley Kubrick,"Drama, Horror",R,146 min,8.4,68,A family heads to an isolated hotel for the wi...,"1,096,441",$44.02M


In [34]:
data.to_csv('Top500_IMDB_MOVIES.csv')